<a href="https://colab.research.google.com/github/chrissintha/Recommender_analysis/blob/main/Recommender__SurpriseLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd

chefmozaccepts=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/chefmozaccepts.csv')

chefmozcuisine=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/chefmozcuisine.csv')

chefmozhours4=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/chefmozhours4.csv')

chefmozparking=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/chefmozparking.csv')

geoplaces2=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/geoplaces2.csv')

rating_final=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/rating_final.csv')

usercuisine=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/usercuisine.csv')

userpayment=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/userpayment.csv')

userprofile=pd.read_csv('https://raw.githubusercontent.com/chrissintha/RestaurantRecommender/main/csv_food/userprofile.csv')

In [34]:
rating_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 45.5+ KB


In [35]:
rating_final['placeID'] = rating_final['placeID'].astype(str)

In [36]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import pandas as pd
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold

SVD

In [38]:
algo = SVD()
rating = rating_final.filter(['userID','placeID', 'rating'])

In [39]:
reader = Reader(line_format='user item rating', rating_scale=(0, 2))

In [40]:
rating_final.set_index('userID')

,placeID,rating,food_rating,service_rating
userID,,,,
U1077,135085,2,2,2
U1077,135038,2,2,1
U1077,132825,2,2,2
U1077,135060,1,2,2
U1068,135104,1,1,2
...,...,...,...,...
U1043,132630,1,1,1
U1011,132715,1,1,0
U1068,132733,1,1,0


In [41]:

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(rating_final[['userID', 'placeID', 'rating']], reader)

In [42]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# We can now use this dataset as we please, e.g. calling cross_validate


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6730  0.6429  0.6707  0.6701  0.7020  0.6717  0.0187  
MAE (testset)     0.5707  0.5570  0.5707  0.5554  0.6006  0.5709  0.0162  
Fit time          0.06    0.06    0.05    0.05    0.05    0.06    0.01    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.05789542198181152,
  0.0648188591003418,
  0.0502476692199707,
  0.05219388008117676,
  0.05247330665588379),
 'test_mae': array([0.57068179, 0.55702605, 0.57067802, 0.5554269 , 0.60062191]),
 'test_rmse': array([0.67301512, 0.64288067, 0.67068265, 0.67010199, 0.70198864]),
 'test_time': (0.0027544498443603516,
  0.002859830856323242,
  0.001844167709350586,
  0.0016140937805175781,
  0.0015892982482910156)}

In [43]:
from surprise import NormalPredictor
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.0012385845184326172, 0.0007843971252441406),
 'test_mae': array([0.82434665, 0.79069324]),
 'test_rmse': array([1.01084858, 0.98211041]),
 'test_time': (0.008124113082885742, 0.0037958621978759766)}

In [44]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.7224


0.7224142575269296

In [45]:
predictions = algo.fit(trainset).test(testset)

Train on a whole trainset and the predict() method

In [46]:
from surprise import KNNBasic
from surprise import Dataset

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [47]:
userID = str('U1077')  # raw user id (as in the ratings file). They are **strings**!
placeID = str(132660)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(userID, placeID, r_ui=4, verbose=True)

user: U1077      item: 132660     r_ui = 4.00   est = 1.20   {'was_impossible': True, 'reason': 'Not enough neighbors.'}


KFold(n_splits=3)

In [48]:
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.6680
RMSE: 0.6923
RMSE: 0.7117


GridSearchCV

In [49]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.7147362803662592
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [50]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

KNN (prediction algorithms)

In [51]:
from surprise import KNNBasic
algo = KNNBasic()

In [52]:
from surprise import BaselineOnly
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

Using ALS


In [53]:
print('Using SGD')
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)

Using SGD


In [54]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               }
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(bsl_options=bsl_options, sim_options=sim_options)

Similarity measure configuration

In [55]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)

In [56]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

Predictions

In [57]:
# Build an algorithm, and train it.
algo = SVD()
algo.fit(trainset)
userID = str('U1077')  # raw user id (as in the ratings file). They are **strings**!
placeID = str(132660)  # raw item id (as in the ratings file). They are **strings**!
# get a prediction for specific users and items.
pred = algo.predict(userID, placeID)
pred

Prediction(uid='U1077', iid='132660', r_ui=None, est=1.232749985999547, details={'was_impossible': False})

In [58]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7095  0.6846  0.6755  0.6951  0.6853  0.6900  0.0116  
MAE (testset)     0.6133  0.5803  0.5817  0.6092  0.5808  0.5931  0.0149  
Fit time          0.00    0.01    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.0040721893310546875,
  0.005808591842651367,
  0.0038635730743408203,
  0.0036420822143554688,
  0.00351715087890625),
 'test_mae': array([0.61330782, 0.5802939 , 0.58166403, 0.60920263, 0.58080377]),
 'test_rmse': array([0.70949928, 0.68462622, 0.67551675, 0.6951349 , 0.68528421]),
 'test_time': (0.0024874210357666016,
  0.0018439292907714844,
  0.0018384456634521484,
  0.0017650127410888672,
  0.0016207695007324219)}

In [59]:
rating_final1 = rating_final.set_index('userID')


In [60]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 2))

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.0013234615325927734, 0.0014569759368896484),
 'test_mae': array([0.80728934, 0.7885392 ]),
 'test_rmse': array([0.9990907 , 0.97328572]),
 'test_time': (0.007975339889526367, 0.0074465274810791016)}

How to get the top-N recommendations for each user

In [61]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for userID, placeID, true_r, est, _ in predictions:
        top_n[userID].append((placeID, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for userID, rating in top_n.items():
        rating.sort(key=lambda x: x[1], reverse=True)
        top_n[userID] = rating[:n]

    return top_n


# First train an SVD algorithm on the  dataset.

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(rating_final[['userID', 'placeID', 'rating']], reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for userID, rating in top_n.items():
   #print(userID, [placeID for (placeID, _) in rating])
  count=0
  if count >10 :
      data = {'userID': [userID], 'placeID': [placeID for (placeID, _) in rating]}
      user_recom = pd.DataFrame.append(data)
      user_recom.to_csv("user_rec.csv",index=False)
      count+=1
#top_n =pd.DataFrame(top_n)

#top_n.to_csv("user_based.csv",index=False)

# New section

precision@k and recall@k

In [62]:
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for userID, _, true_r, est, _ in predictions:
        user_est_true[userID].append((est, true_r))
        precisions = dict()

    recalls = dict()
    for userID, rating in user_est_true.items():

        # Sort user ratings by estimated value
        rating.sort(key=lambda x: x[0], reverse=True)
         
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in rating)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in rating[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in rating[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[userID] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[userID] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

    
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Item Based

In [63]:
uid = str(rating_final['userID'])  # raw user id (as in the ratings file). They are **strings**!
iid = str(rating_final['placeID'])  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
p_useritem = algo.predict(uid, iid, r_ui=4, verbose=True)
p_useritem

user: 0       U1077
1       U1077
2       U1077
3       U1077
4       U1068
        ...  
1156    U1043
1157    U1011
1158    U1068
1159    U1068
1160    U1068
Name: userID, Length: 1161, dtype: object item: 0       135085
1       135038
2       132825
3       135060
4       135104
         ...  
1156    132630
1157    132715
1158    132733
1159    132594
1160    132660
Name: placeID, Length: 1161, dtype: object r_ui = 4.00   est = 1.21   {'was_impossible': False}


Prediction(uid='0       U1077\n1       U1077\n2       U1077\n3       U1077\n4       U1068\n        ...  \n1156    U1043\n1157    U1011\n1158    U1068\n1159    U1068\n1160    U1068\nName: userID, Length: 1161, dtype: object', iid='0       135085\n1       135038\n2       132825\n3       135060\n4       135104\n         ...  \n1156    132630\n1157    132715\n1158    132733\n1159    132594\n1160    132660\nName: placeID, Length: 1161, dtype: object', r_ui=4, est=1.209903121636168, details={'was_impossible': False})